## Mannequin Challenge Dataset Downloader
This notebook is a downloader for Mannequin Challenge Dataset. Written by Myeong-Gyu.Lee

* Reference: https://blog.naver.com/PostView.nhn?blogId=skyshin0304&logNo=221620513883&proxyReferer=https:%2F%2Fwww.google.com%2F

type `pip install pytube3` to install pytube library.

### Test single video download

In [1]:
from pytube import YouTube
from tqdm import tqdm
import os, cv2, shutil, math, datetime, ast, warnings
import matplotlib.pyplot as plt
import pandas as pd

warnings.filterwarnings("ignore",category=DeprecationWarning)

# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore",category=DeprecationWarning)
#     import md5, sha

%matplotlib inline

In [2]:
_url = 'https://www.youtube.com/watch?v=KMtgexdtcGY'
yt = YouTube(_url)
print("영상 제목 :", yt.title)
print("영상 길이 :", yt.length)
print("영상 평점 :", yt.rating)
print("영상 썸네일 링크: ", yt.thumbnail_url)
print("영상 조회수 :", yt.views)
print("영상 설명 :", yt.description)

영상 제목 : YouTube
영상 길이 : 203
영상 평점 : 5.0
영상 썸네일 링크:  https://i.ytimg.com/vi/KMtgexdtcGY/maxresdefault.jpg
영상 조회수 : 200
영상 설명 : The FHS Wind Ensemble took on the Mannequin Challenge! Not originally my video but it was too fun NOT to upload, and all credit is given! :)

See if you can spot me!
---------------------------------
Twitter ~ https://twitter.com/JessicaWitXx
Instagram ~ https://www.instagram.com/jesswitxx/
Snapchat ~ jessicawitxx
Vlog Channel ~ https://goo.gl/6ZhRuu
Collab Channel ~ https://goo.gl/rHNw96
---------------------------------
Song 1:
Merry Go Slower by Kevin MacLeod is licensed under a Creative Commons Attribution license (https://creativecommons.org/licenses/by/4.0/)
Source: http://incompetech.com/music/royalty-free/index.html?isrc=USUAN1100732
Artist: http://incompetech.com/
Song 2:
Divertimento K131 by Kevin MacLeod is licensed under a Creative Commons Attribution license (https://creativecommons.org/licenses/by/4.0/)
Source: http://incompetech.com/music/royalty-

In [3]:
_yt_streams = yt.streams
print("다운가능한 영상 상세 정보 :")
for i, stream in enumerate(_yt_streams.all()):
    print(i, " : ", stream)

다운가능한 영상 상세 정보 :
0  :  <Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">
1  :  <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">
2  :  <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
3  :  <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="video">
4  :  <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d4014" progressive="False" type="video">
5  :  <Stream: itag="244" mime_type="video/webm" res="480p" fps="30fps" vcodec="vp9" progressive="False" type="video">
6  :  <Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">
7  :  <Stream: itag="243" mime_type="video/webm" res="360p" fps="30fps" vcodec="v

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  This is separate from the ipykernel package so we can avoid doing imports until


### Build the dataframe and sort by `res` to get highest resolution video.
Get only `video/mp4` rows.

In [4]:
def get_download_info(yt_streams, video_url):
    stream_df_list = []
    
    for stream in yt_streams:
        try:
            stream_dict = dict()
            stream_str = str(stream)
            stream_elements = stream_str.replace('Stream: ', '').replace('=', ':').replace('<', '').replace('>', '').replace('"', '').split(' ')
            for elemnt in stream_elements:
                stream_dict[elemnt.split(':')[0]] = elemnt.split(':')[-1]
            stream_df_list.append(pd.DataFrame.from_dict(stream_dict, orient='index').T)
        except:
            print("This yt_stream does not have any downloadable streams.")

    stream_df_global = pd.concat(stream_df_list)
    stream_df_global['videoID'] = str(video_url.split('/')[-1].split('=')[-1])
    stream_df_global.set_index('videoID', inplace = True)
    stream_df_global = stream_df_global[pd.notnull(stream_df_global['res'])]
    stream_df_global['res'] = stream_df_global['res'].str.replace(pat=r'[A-Za-z]', repl= r'', regex=True)
    stream_df_global = stream_df_global.astype({'itag': int, 'res': int})
    stream_df_global = stream_df_global.sort_values(by='res', ascending=False)
    stream_df_global = stream_df_global[stream_df_global['mime_type'] == 'video/mp4']
    
    # itag를 이용해 가장 높은 해상도의 Video Download
    highest_stream = yt_streams.get_by_itag(stream_df_global.iloc[0]['itag'])
    
    return stream_df_global, highest_stream

In [5]:
stream_df_global, highest_stream = get_download_info(_yt_streams, _url)
print('Highest resolution stream info:', highest_stream)
stream_df_global.head(3)

Highest resolution stream info: <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">


,itag,mime_type,res,fps,vcodec,acodec,progressive,type,abr
videoID,,,,,,,,,
KMtgexdtcGY,22,video/mp4,720,30fps,avc1.64001F,mp4a.40.2,True,video,NaN
KMtgexdtcGY,136,video/mp4,720,30fps,avc1.4d401f,NaN,False,video,NaN
KMtgexdtcGY,135,video/mp4,480,30fps,avc1.4d4014,NaN,False,video,NaN


In [6]:
print("선택된 stream 다운로드:", highest_stream)
# highest_stream.download(output_path='D:/MannequinChallenge_Videos', filename=stream_df_global.index[0])

선택된 stream 다운로드: <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">


### Read whole `.txt` file and download

In [7]:
meta_txt_path = 'D:/MannequinChallenge'

Get video url and build combined dictionary

In [8]:
entire_video_urls = {}

for path in os.listdir(meta_txt_path):
    path_url_list = []
    
    txt_path = os.path.join(meta_txt_path, path)
    for txt in os.listdir(txt_path):
        f = open(os.path.join(txt_path, txt))
        line = f.readline().replace('\n', '')
        path_url_list.append(line)
    entire_video_urls.update({path:list(set(path_url_list))})

Download videos.

In [9]:
failed_video_urls = {}

for key, value in entire_video_urls.items():
    failure_url_list = []
    for video_url in tqdm(value):
        try:
            filename = video_url.split('=')[-1]
            output_path = os.path.join('D:/MannequinChallenge_Videos', key)
            os.makedirs(output_path, exist_ok=True)
            if not os.path.exists(os.path.join(output_path, filename+'.mp4')):
                yt_streams = YouTube(video_url)
                _yt_str = yt_streams.streams
                stream_df_global, highest_stream = get_download_info(_yt_str, video_url)
                highest_stream.download(output_path=output_path, filename=filename)
            else:
                print("This stream is exist: {}".format(filename+'.mp4'))
                continue
        except:
            failed_url = video_url.split('=')[-1]
            print("This stream is not downloadable: {}".format(failed_url))
            failure_url_list.append(failed_url)
            continue
        failed_video_urls.update({key:failure_url_list})

  0%|                                                                                          | 0/173 [00:00<?, ?it/s]

This stream is exist: _qcR2QMFvAY.mp4
This stream is exist: zf2y_skLD1c.mp4
This stream is exist: Acd0zgZxOs0.mp4
This stream is exist: fo1IiDRMGxk.mp4


  3%|██▎                                                                               | 5/173 [00:01<01:00,  2.79it/s]

This stream is not downloadable: HLb60swvg6c
This stream is exist: bOR0VLdZcMA.mp4
This stream is exist: 7_KcIVkSBd8.mp4
This stream is exist: 9KIAuXOEzoc.mp4
This stream is exist: 07j4NKBs8jQ.mp4
This stream is exist: dJVMqpY3B98.mp4
This stream is exist: ghGyRHHdhJI.mp4
This stream is exist: L4-TDThGgRM.mp4
This stream is exist: GCWA___531Y.mp4


  8%|██████▌                                                                          | 14/173 [00:03<00:48,  3.26it/s]

...

 80%|█████████████████████████████████████████████████████████████████▌                | 20/25 [02:58<00:20,  4.19s/it]

This stream is not downloadable: YMSmsPx5wlw


 88%|████████████████████████████████████████████████████████████████████████▏         | 22/25 [03:24<00:22,  7.62s/it]

This stream is not downloadable: zeEWn2n99U4


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [04:26<00:00, 10.67s/it]


In [13]:
print("Done downloading videos! \nFailure video summary: \n - train: {}, - val: {}, - test: {}".format(
                len(failed_video_urls['train']), 
                len(failed_video_urls['validation']),
                len(failed_video_urls['test'])))

Done downloading videos! 
Failure video summary: 
 - train: 217, - val: 11, - test: 28
